##Projekt 2 - Steam Sale!

Beautifulsoup och Pandas

In [322]:
#vi börjar med perus imports av nödvändiga moduler
from bs4 import BeautifulSoup as bs
from os.path import exists
import pandas as pd
import requests, datetime, re

Här lagar vi en dictionary och söker dom 5 första sidorna med erbjudanden från steam. Vi loopar in filerna in i en dictionary för EZ access.

In [323]:
#toma listor för all data
name,rating,reviews,sale,price,ogprice,release,win,lin,OSX,other,time = ([] for i in range(12))
#dictionary för att konvertera data till en dataframe senare
datadict = {"Spelnamn": name,"Rating": rating,"#Reviews" : reviews,"Rabatt%" :sale,"Pris":price,"OrdinariePris":ogprice,"Utgivningsår":release,"Win":win,"Lin":lin,"OSX":OSX,"Other":other,"Tid":time}
#rådata för ratings tas först in i en egen lista
ratingraw = []
x = datetime.datetime.now()
#För fulla poäng krävs att man samlar data från 5 sidor med erbjudanden.
#jag har testat att min loop lyckas åtminstone parse dom 100 första sidorna.
#några datapunkter fick min orginalkod för endast de 5 första sidorna att crasha
#men med några ändringar så funkar de 100 första(in my testing)
for i in range(1,101):
  r = requests.get("https://store.steampowered.com/search/?specials=1&page="+str(i))
  soppa = bs(r.content)
  for a in soppa.findAll('a',  {'class':'search_result_row ds_collapse_flag '}):
    name.append(a.find('span', {'class': 'title'}).text)
    #vi tar in först bara span taggen med reviews, och så hanterar vi den senare var vi söker fram antal reviews och dess positivitet/negativitet
    ratingtext = a.find('span', {'class': ['search_review_summary positive','search_review_summary mixed','search_review_summary negative'] })
    if ratingtext != None:
      ratingraw.append(ratingtext['data-tooltip-html'])
    else:
      #ifall spelet inte har reviews
      ratingraw.append("No user reviews")
    #hitta erbjudandets procent
    salediv = a.find('div', {'class': 'col search_discount responsive_secondrow'})
    if salediv.span != None:
      sale.append(salediv.span.text)
    else:
      #vissa spel som är free to play dyker ändå upp i queryn, så måste sätta in data för denna edgecase
      sale.append("Game is free to play")
    pricediv =  a.find('div', {'class': 'col search_price_discount_combined responsive_secondrow'})
    if pricediv != None:
      if pricediv['data-price-final'] != "0":
        #konvertera priset till float
        price.append(float(pricediv['data-price-final'])/100)
        ogprice.append(float(a.find('div', {'class': 'col search_price discounted responsive_secondrow'}).strike.text.strip('$').replace(',','')))
      else:
        #free to play spel, vi lägger 0.0 som pris så det matchar resten av data
        price.append(float(pricediv['data-price-final']))
        ogprice.append(float(pricediv['data-price-final']))
    releasediv = a.find('div', {'class':'col search_released responsive_secondrow'})
    if releasediv.text != "":
      
      try:
        #konvertera string till datum
        releasedate = datetime.datetime.strptime(releasediv.text, '%b %d, %Y')
        #konvertera till endast år
        release.append(releasedate.strftime("%G"))
      except ValueError:
        #konvertera string till datum ifall förra datumformatet inte matchar
        releasedate = datetime.datetime.strptime(releasediv.text, '%b %Y')
        #konvertera till endast år
        release.append(releasedate.strftime("%G"))
      except:
        #ifall vi hittar något annat datumformat så skippar vi konvertering och sätter in strängen med datumet
        releasedate = releasediv.text
        release.append(releasedate)
          
    else:
      #bubndles har inget releasdatum på sidan
      release.append("No realease data for bundles")
    #hämta spans för vilka operativsystem som stöds, eller om spelet också  innehåller tex ett soundtrack  
    winspan = a.find('span', {'class': 'platform_img win'})
    macspan = a.find('span', {'class': 'platform_img mac'})
    linspan = a.find('span', {'class': 'platform_img linux'})
    otherspan = a.find('span', {'class': ['platform_img music','platform_img streamingvideo']})
    #Använd 0 eller 1 för att beteckna ifall spelet stöder platformen (operativsystem)
    if winspan != None:
      win.append(1)
    else:
      win.append(0)
    if macspan != None:
      OSX.append(1)
    else:
      OSX.append(0)
    if linspan != None:
      lin.append(1)
    else:
      lin.append(0)
    if otherspan != None:
      other.append(1)
    else:
      other.append(0)
    #tid när datan hämtades
    time.append(x.strftime("%c"))



Nu har vi all data samlad, men måste göra lite processing på ratings så vi får ut både mängden reviews och ändrar "very positive, positive" osv. till numror. Utöver det skall vi också gräva ut mänden med recensioner.

In [324]:
#Skapa en sifferbaserad rating enligt alternativen som steam använder i textform (MostlyPositive, Overwhelmingly Positive ...)
ratingsdict = {"Overwhelmingly Negative" : 0,
               "Very Negative" : 1,
               "Negative" : 2,
               "Mostly Negative" : 3,
               "Mixed" : 4,
               "Mostly Positive" : 5,
               "Positive" : 6,
               "Very Positive" : 7,
               "Overwhelmingly Positive" : 8,
               "No user reviews": -1}
               

for i in range(len(ratingraw)):
  x = re.split("<br>",ratingraw[i])
  rating.append(ratingsdict[x[0]])
  if x[0] != "No user reviews":
    #jag är ärlig och erkänner att denna fina regex är lånad från stackowerflow
    numbers = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?",x[1])
    reviews.append(int(numbers[1].replace(",","")))
  else:
    reviews.append("0")


In [325]:
steamDf = pd.DataFrame(datadict)
steamDf

,Spelnamn,Rating,#Reviews,Rabatt%,Pris,OrdinariePris,Utgivningsår,Win,Lin,OSX,Other,Tid
0,Red Dead Redemption 2,7,198892,-50%,29.99,59.99,2019,1,0,0,0,Tue Nov 30 14:50:58 2021
1,The Sims™ 4,7,31144,-88%,4.79,39.99,2020,1,0,0,0,Tue Nov 30 14:50:58 2021
2,Cyberpunk 2077,5,411611,-50%,29.99,59.99,2020,1,0,0,0,Tue Nov 30 14:50:58 2021
3,The Elder Scrolls® Online,7,90897,-70%,5.99,19.99,2017,1,0,1,0,Tue Nov 30 14:50:58 2021
4,Stellaris,7,83917,-75%,9.99,39.99,2016,1,1,1,0,Tue Nov 30 14:50:58 2021
...,...,...,...,...,...,...,...,...,...,...,...,...
2420,Chronos series,7,375,-42%,34.78,59.98,No realease data for bundles,1,0,0,0,Tue Nov 30 14:50:58 2021
2421,ASTRONEER + Soundtrack,7,71257,-37%,22.49,35.98,No realease data for bundles,1,0,0,1,Tue Nov 30 14:50:58 2021
2422,The Thrill of the Fight - VR Boxing,8,2534,-20%,7.99,9.99,2019,1,0,0,0,Tue Nov 30 14:50:58 2021
2423,Trine Enchanted Edition,8,10377,-75%,3.74,14.99,2009,1,1,1,0,Tue Nov 30 14:50:58 2021


In [326]:
#Kom ihåg att granska (och ge mig en bild av datan) genom att använda t.ex. head() 
steamDf.head()

,Spelnamn,Rating,#Reviews,Rabatt%,Pris,OrdinariePris,Utgivningsår,Win,Lin,OSX,Other,Tid
0,Red Dead Redemption 2,7,198892,-50%,29.99,59.99,2019,1,0,0,0,Tue Nov 30 14:50:58 2021
1,The Sims™ 4,7,31144,-88%,4.79,39.99,2020,1,0,0,0,Tue Nov 30 14:50:58 2021
2,Cyberpunk 2077,5,411611,-50%,29.99,59.99,2020,1,0,0,0,Tue Nov 30 14:50:58 2021
3,The Elder Scrolls® Online,7,90897,-70%,5.99,19.99,2017,1,0,1,0,Tue Nov 30 14:50:58 2021
4,Stellaris,7,83917,-75%,9.99,39.99,2016,1,1,1,0,Tue Nov 30 14:50:58 2021


In [327]:
#och describe() för att få en bättre översikt över den insamlade datan.
steamDf.describe()

,Rating,Pris,OrdinariePris,Win,Lin,OSX,Other
count,2425.000000,2425.000000,2425.000000,2425.000000,2425.000000,2425.000000,2425.000000
mean,6.498969,14.012907,34.574124,0.997113,0.261443,0.335258,0.021031
std,1.294225,14.885169,39.894689,0.053661,0.439511,0.472178,0.143517
min,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,5.070000,14.990000,1.000000,0.000000,0.000000,0.000000
50%,7.000000,9.990000,24.990000,1.000000,0.000000,0.000000,0.000000
75%,7.000000,17.580000,39.990000,1.000000,1.000000,1.000000,0.000000
max,8.000000,248.630000,756.400000,1.000000,1.000000,1.000000,1.000000


Sedan efter att vi fått vår dataframe så ska vi bygga upp vår .csv fil.
Vi börjar med att kolla om filen redan existerar, om inte, skapar vi en fil.

In [328]:
if exists("/content/steamsale.csv"):
  #vi lämnar bort index, så vi inte får en massa siffror i vår .csv och ifallet var filen redan finns så lämnar vi också bort headers så de inte söndrar data
  steamDf.to_csv('steamsale.csv', mode='a', header=False, index=False)
else:
  #vi lämnar bort index, så vi inte får en massa siffror i vår .csv 
  steamDf.to_csv('steamsale.csv', index=False)